# Macros

Macros and preproccessors are very important in C/C++. They are less important in Rust, and have more features, which we will look at them here.

You have seen macros before, for example, `println!()` is a macro:

In [2]:
println!("Hello {}", 2 + 2);

Hello 4


You can print without macro as well, it's just more verbose:

In [11]:
use std::io::{self, Write};

io::stdout().write_all(b"Hello ").unwrap();
io::stdout().write_all((2 + 2).to_string().as_bytes()).unwrap(); // `println!` is more efficient here
io::stdout().write_all(b"\n").unwrap();


Hello 4


## Macros that are not needed in Rust

In C/C++ macros are used for some basic works. For example, organizing code in C/C++ is by `#include` macro, but in Rust we have a modern module system. But if you really need including some code from another file (it can be useful in some rare scenarios, like including a generated code) there is an `include!` macro:

In [21]:
std::fs::write("/tmp/some_code.rs", "2 + 3").unwrap();

In [22]:
include!("/tmp/some_code.rs")

5

We can also include other files as a string literal with `include_str!`, which is more useful, like for including a config. The equivalent in C is `#embed`, which is added in C23.

In [23]:
let x: &'static str = include_str!("/tmp/some_code.rs");
x

"2 + 3"

Or as a byte array with `include_bytes!`, for non utf8 files, or assets like images:

In [24]:
let x: &'static [u8; 5] = include_bytes!("/tmp/some_code.rs");
x

[50, 32, 43, 32, 51]

Another type of macros useful in C but not needed in Rust and C++ is `#define CONSTANT value`. Rust supports what C++ calls `constexpr`:

In [2]:
const FOO: i32 = 5;
FOO

5

`const`s are just values evaluated at compile time. They don't have address in memory, and will be copied on each usage. We can use `const`s even in types:

In [4]:
const FOO: usize = 2 + 3;
let x: [u8; FOO] = [1, 2, 3, 4, 5];
x

[1, 2, 3, 4, 5]

Which doesn't work for variables, since compiler needs type information to generate code:

In [5]:
let foo = 2 + 3;
let x: [u8; foo] = [1, 2, 3, 4, 5];
x

Error: attempt to use a non-constant value in a constant

Error: consider using `const` instead of `let`

Compiler needs knowing `const` values at compile time, so they can't be initialized with arbitary functions:

In [6]:
const FOO: &str = get_something_from_network();

fn get_something_from_network() -> &'static str {
    "hello" // nothing prevents this function to actually get result from network
    // and Rust only decide from signature, because of semver.
}

Error: cannot call non-const fn `get_something_from_network` in constants

But it can call `const fn`s:

In [8]:
{
    const FOO: &str = a_compile_time_evaluatable_function();

    const fn a_compile_time_evaluatable_function() -> &'static str {
        "hello"
    }

    FOO
}

"hello"

`const fn` can not call non const fn:

In [10]:
const fn not_compile_time_evaluatable_function() -> &'static str {
    get_something_from_network()
}

fn get_something_from_network() -> &'static str {
    "hello" // nothing prevents this function to actually get result from network
}

Error: cannot call non-const fn `get_something_from_network` in constant functions

But it can do fairly complex things:

In [14]:
const fn const_fibo(n: usize) -> usize {
    let mut i = 0;
    let mut a = 1;
    let mut b = 1;
    while i < n { // for is not available (yet) in const fn, since iterator methods are not const fn
        i += 1;
        (a, b) = (b, a + b);
    }
    a
}

[2; const_fibo(5)]

[2, 2, 2, 2, 2, 2, 2, 2]

Many functions in standard library are `const fn`:

In [19]:
{
    const FOO: bool = Some(12).is_some(); // `.is_some` is `const fn`
    FOO
}

true

Const context are used in other places in the language as well. For example, `static` initializors should be `const fn` since Rust can't run runtime code before the `main` function.

That's enough `const` for now. Let's get back to macros. All macros we have seen was in form `macro!()` or `macro![]` (for vec). The thing is, `()`, `[]`, `{}` are equal for macros:

In [23]:
println!["{:?}", vec!(1, 2, 3)]; // don't write this in real code!
vec!{
    10,
    20,
    30,
}

[1, 2, 3]


[10, 20, 30]

And there is another, completely different shape of macros, called attribute macros. For example, in C we can write `#ifdef windows` to define a function only on windows. In Rust, we can do it with `#[cfg(windows)]`:

In [25]:
#[cfg(windows)]
fn some_fn() {
    println!("in windows");
}

#[cfg(not(windows))]
fn some_fn() {
    println!("not in windows");
}

some_fn();

not in windows


`cfg` has also an expression variant, which evaluates to true or false:

In [26]:
(cfg!(windows), cfg!(not(windows)))

(false, true)

So we can write above code in this way:

In [29]:
fn some_fn() {
    if cfg!(windows) {
        println!("in windows");
    } else {
        println!("not in windows");
    }
}

some_fn();

not in windows


`#[cfg]` actually removes code but `cfg!` just evaluates to `true` and `false` and relies on optimizer to remove dead code. So this works on non windows:

In [31]:
#[cfg(windows)]
fn some_fn() {
    some_non_existent_fn();
}

#[cfg(not(windows))]
fn some_fn() {
    println!("not in windows");
}

some_fn();

not in windows


But this will fail:

In [32]:
fn some_fn() {
    if cfg!(windows) {
        some_non_existent_fn();
    } else {
        println!("not in windows");
    }
}

some_fn();

Error: cannot find function `some_non_existent_fn` in this scope

So most of the time you need `#[cfg]` because you need to call function which themself are `#[cfg]` ed.

## User defined macros

Rust has two entirely different ways to define a macro. Declarative macros and procedural macros. Here is a simple declarative macro:

In [33]:
// This is a simple macro named `say_hello`.
macro_rules! say_hello {
    // `()` indicates that the macro takes no argument.
    () => {
        // The macro will expand into the contents of this block.
        println!("Hello!")
    };
}

say_hello!();

Hello!


A little more complex macro:

In [39]:
macro_rules! calculate {
    (eval $e:expr) => {
        {
            let val: usize = $e; // Force types to be integers
            println!("{} = {}", stringify!{$e}, val);
        }
    };
}

calculate! {
    eval 1 + 2
}

calculate! {
    eval (1 + 2) * (3 / 4)
}

// stringify is not special, its just a macro
stringify!(1 2    (3) hello . goodbye)

1 + 2 = 3
(1 + 2) * (3 / 4) = 0


"1 2 (3) hello.goodbye"

Rust macros has several benefit over C macros. First, their result is abstract syntax tree, not a simple string, so things like this is not possible:

In [40]:
macro_rules! end_paren {
    () => {
        )
    };
}

Error: unexpected closing delimiter: `}`

Error: mismatched closing delimiter: `)`

Second, Rust macros are hygienic. That means macro internals can not use bindings in the scope and vise versa:

In [41]:
macro_rules! using_a {
    ($e:expr) => {
        {
            let a = 42;
            $e + a
        }
    }
}

let a = 5;
using_a!(a + 3)

50

And this will get compile error:

In [42]:
macro_rules! using_a {
    ($e:expr) => {
        {
            $e + a
        }
    }
}

let a = 5;
using_a!(a + 3)

Error: cannot find value `a` in this scope

Error: unused macro definition: `say_hello`

Error: unused macro definition: `calculate`

Declaritive macros are restricted to `macro!`, but procedural macros support writing attribute macros as well. For defining a proc macro, you need to have a crate with type `proc_macro` which compiler compiles it and execute it to find out what is the result of macro. We can't have multiple crates in jupyter notebook, but to make a sense of how proc macros work, we can use `proc_macro2` crate:

In [8]:
:dep proc-macro2 = "1"
:dep syn = { version = "1", features = ["full"] }
:dep quote = "1"

A proc macro is a rust function which recieves a token stream (output of Rust's lexer) and returns a token stream.

In [6]:
use proc_macro2::TokenStream;
use quote::quote;

fn a_proc_macro_like_fn(input: TokenStream) -> TokenStream {
    dbg!(&input);
    let count_of_tokens = input.into_iter().count();
    quote! {
        let result = #count_of_tokens;
    }
}

a_proc_macro_like_fn(quote!(2 + (3 + 5 + 7)))

[src/lib.rs:5] &input = TokenStream [
    Literal {
        lit: 2,
    },
    Punct {
        char: '+',
        spacing: Alone,
    },
    Group {
        delimiter: Parenthesis,
        stream: TokenStream [
            Literal {
                lit: 3,
            },
            Punct {
                char: '+',
                spacing: Alone,
            },
            Literal {
                lit: 5,
            },
            Punct {
                char: '+',
                spacing: Alone,
            },
            Literal {
                lit: 7,
            },
        ],
    },
]


TokenStream [Ident { sym: let }, Ident { sym: result }, Punct { char: '=', spacing: Alone }, Literal { lit: 3usize }, Punct { char: ';', spacing: Alone }]

Token stream knows about group of parenthesis, but it doesn't parse the input. `quote` is a third party crate which allows us to return token streams with a beatiful api. `quote! { let result = #count_of_tokens; }` means get `#count_of_tokens` from the variable with the same name (which in our case is 3) and then generate token stream equivalent of `let result = 3;`.

For doing anything complex, you nead a Rust parser, `syn` provides a Rust parser, mainly for proc macros. We can use that to write a function which sums up all fields of a struct:

In [21]:
use syn::{parse2, ItemStruct};

fn a_proc_macro_that_sum_fields(input: TokenStream) -> TokenStream {
    let st: ItemStruct = parse2(input).unwrap();
    let name = &st.ident;
    let field_ident = st.fields.iter().filter_map(|x| x.ident.as_ref());
    quote! {
        #st // include input itself, to not need to define struct twice

        impl #name {
            fn summer(&self) -> i32 {
                0 #( + self.#field_ident )*
            }
        }
    }
}

a_proc_macro_that_sum_fields(quote! {
    struct X {
        field1: i32,
        field2: i32,
        hello: i32,
    }
}).to_string()

"struct X { field1 : i32 , field2 : i32 , hello : i32 , } impl X { fn summer (& self) -> i32 { 0 + self . field1 + self . field2 + self . hello } }"

Now you have a sense of how proc macros work. Proc macros are very popular in Rust ecosystem, specially for derive macros, bindgen, and async. 52% of all published crates in crates.io depend indirectly on `syn`.